In [ ]:
import os
from crewai import Agent, Task, Crew
from crewai.process import Process


# 1. Initialize the Language Model (LLM)
# It is assumed that the OPENAI_API_KEY environment variable is set.
# ChatOpenAI uses the environment variable automatically.
# You can replace this with any other supported LLM by CrewAI/LangChain,
# such as Groq, Mistral, Ollama, etc.

llm = ChatOpenAI(model="gpt-4o-mini")

# 2. Define the Agent(s)

researcher = Agent(
    role='Senior Research Analyst',
    goal='Discover and summarize the latest trends in renewable energy',
    backstory='An expert in sustainability and market analysis. Meticulous and detail-oriented.',
    verbose=True,
    allow_delegation=False,
    llm=llm  # Inject the initialized LLM here
)

# 3. Define the Task(s)

research_task = Task(
    description=(
        "Analyze the current global news and market data for renewable energy. "
        "Specifically focus on **solar panel efficiency** improvements and **battery storage** "
        "innovations over the last six months. "
        "The final output must be a concise, three-paragraph summary."
    ),
    expected_output='A three-paragraph summary report on solar and battery advancements.',
    agent=researcher  # Assign the task to the defined agent
)

# 4. Create the Crew and Run the Process

# For simplicity, this example uses a single agent and a single task.
# A full crew typically has multiple agents and multiple tasks.
project_crew = Crew(
    agents=[researcher],
    tasks=[research_task],
    process=Process.sequential,  # Execute tasks in order
    verbose=2  # High verbosity for observing the agent's thought process
)

print("### Crew Starting Execution ###")

# Prompting the LLM is implicitly handled when the crew is kicked off,
# as the agents use the LLM to process their assigned tasks and goals.
result = project_crew.kickoff()

# 5. Output the Result

print("\n\n### Task Execution Complete ###")
print(f"Final Report:\n{result}")

In [ ]:
"""
Extract the claims and return a JSON for each claim retrieved from the argument.
"""

In [ ]:
import os
from crewai import Agent, Task, Crew, LLM
from crewai.process import Process

# Ensure the environment variable is set before running:
# export GEMINI_API_KEY="YOUR_API_KEY"

# 1. Initialize the LLM for Gemini
# CrewAI uses a simplified LLM class that interfaces with LiteLLM.
# The model name must be provided in the format: <provider>/<model_name>
# Common Gemini models: gemini/gemini-2.5-flash, gemini/gemini-2.5-pro

from dotenv import load_dotenv

load_dotenv()

gemini_llm = LLM(
    model='gemini/gemini-2.5-pro',
    # LiteLLM automatically uses GEMINI_API_KEY from environment
    temperature=0.1
)

# 2. Define the Agent, injecting the Gemini LLM
gemini_researcher = Agent(
    role='AI Configuration Specialist',
    goal='Provide a concise, detailed report on Gemini LLM integration into CrewAI.',
    backstory='An expert in multi-agent orchestration frameworks and Google AI APIs. Highly accurate and focused on syntax.',
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm # <-- Inject the configured Gemini LLM here
)

# 3. Define the Task
integration_task = Task(
    description=(
        "Summarize the Python steps required to initialize a CrewAI Agent using the 'gemini/gemini-2.5-flash' model. "
        "The summary must explicitly mention the required environment variable for the API key."
    ),
    expected_output='A four-step guide covering installation, API key variable, LLM initialization syntax, and Agent injection.',
    agent=gemini_researcher
)

# 4. Create and Run the Crew
project_crew = Crew(
    agents=[gemini_researcher],
    tasks=[integration_task],
    process=Process.sequential,
    verbose=1
)

print("### Crew Starting Execution with Gemini ###")

result = project_crew.kickoff()

print("\n\n### Gemini Crew Task Complete ###")
print(f"Final Report:\n{result}")

In [ ]:
import os
import json
from crewai import Agent, Task, Crew, LLM
from crewai.process import Process
from dotenv import load_dotenv
from pydantic import BaseModel, Field
from typing import List

# --- 1. CONFIGURATION AND SCHEMA ---

# Load environment variables from .env file
load_dotenv()

# Define the structured output schema using Pydantic
class Claim(BaseModel):
    """Represents a single verifiable claim extracted from the text."""
    claim_id: int = Field(description="A unique integer ID for this claim, starting from 1.")
    statement: str = Field(description="The exact claim or assertion made in the text.")
    confidence_score: float = Field(description="A score from 0.0 to 1.0 indicating the certainty of the claim's existence in the text.")
    verifiable: bool = Field(description="True if the claim contains factual, potentially provable information; False if it is an opinion, definition, or prediction.")

class ClaimList(BaseModel):
    """The final list structure for all claims extracted."""
    claims: List[Claim] = Field(description="A list containing all claims extracted from the input text.")


# --- 2. LLM SETUP ---

# Initialize the Gemini LLM
# LiteLLM automatically uses the GEMINI_API_KEY environment variable.
gemini_llm = LLM(
    model='gemini/gemini-2.5-flash',
    temperature=0.0, # Low temperature for reliable extraction and JSON generation
    # Setting max_retries ensures robustness in structured output generation
    max_retries=3 
)

# --- 3. AGENT DEFINITION ---

claim_extractor_agent = Agent(
    role='Structured Claim Extractor',
    goal='Accurately identify all verifiable claims within a given text and structure them according to the required JSON schema.',
    backstory='You are a meticulous language model specializing in parsing unstructured text into clean, valid JSON format. You are highly reliable in detecting subtle assertions and assigning accurate confidence scores.',
    verbose=True,
    allow_delegation=False,
    llm=gemini_llm
)

In [ ]:
# --- 4. TASK DEFINITION ---

INPUT_TEXT = """
🔥 BREAKING – UNBELIEVABLE!!! 🔥
Just came across this mind-blowing photo — all the biggest tech bosses hanging out together over Thanksgiving dinner! 😲🦃

There’s Elon Musk, Mark Zuckerberg, Sundar Pichai, Satya Nadella, Tim Cook — sitting around a table like old friends, laughing, talking, drinks on the table. 🍷🍽️
Someone is saying this was snapped right after a secret “super-AI summit” where they met to decide the future of AI & Web. 💡🤖

It’s “the most powerful dinner” ever — feels crazy that these guys, who usually fight behind closed doors, are chilling together! 😮

If this image is real, it means something big is cooking 👀… share this around so everyone sees! 🌐🔥
"""

extraction_task = Task(
    description=(
        f"Analyze the following text and extract all distinct, factual claims. "
        f"Ignore opinions, definitions, or predictions unless they contain a concrete, measurable assertion.\n\n"
        f"TEXT TO ANALYZE:\n---\n{INPUT_TEXT}\n---"
    ),
    expected_output="A list of claims strictly formatted as a JSON array conforming to the Pydantic ClaimList schema.",
    agent=claim_extractor_agent,
    # Crucial step: enforce structured output using the Pydantic model
    output_pydantic=ClaimList
)

# --- 5. CREW EXECUTION ---

claim_crew = Crew(
    agents=[claim_extractor_agent],
    tasks=[extraction_task],
    process=Process.sequential,
    verbose=1
)

# Execute the crew
print("--- Starting Claim Extraction ---")
crew_result = claim_crew.kickoff()

# --- 6. OUTPUT PROCESSING ---

# Access the structured Pydantic output directly
if crew_result.tasks_output and hasattr(crew_result.tasks_output[0], 'pydantic'):
    final_claim_list: ClaimList = crew_result.tasks_output[0].pydantic
    
    print("\n--- Final Structured JSON Output ---")
    # Convert the Pydantic model to a pretty JSON string
    print(json.dumps(final_claim_list.model_dump(), indent=4))
    
    print("\n--- Programmatic Access Example ---")
    for claim in final_claim_list.claims:
        print(f"ID: {claim.claim_id} | Verifiable: {claim.verifiable} | Statement: {claim.statement}")
else:
    print("\n--- Error: Failed to retrieve structured Pydantic output. ---")